In [1]:
import numpy  as np
import pandas as pd
from metrics import MSE
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
X_train = pd.read_pickle("../../../../Datasets/temporal/final/X_train.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"])
c = X_train.columns.tolist()
X_train = X_train.to_numpy()
X_val = pd.read_pickle("../../../../Datasets/temporal/final/X_val.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"]).to_numpy()
X_test = pd.read_pickle("../../../../Datasets/temporal/final/X_test.pkl").drop(columns=["gPlusUserId", "gPlusPlaceId"]).to_numpy()
y_train = pd.read_pickle("../../../../Datasets/temporal/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../../../../Datasets/temporal/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../../../../Datasets/temporal/final/y_test.pkl").to_numpy()

In [3]:
def build_and_compile_model():
    model = tf.keras.Sequential([
        # tf.keras.layers.Dense(512, activation=tf.nn.relu),
        # tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.Dense(256, activation=tf.nn.relu),
        # tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.Dense(64, activation=tf.nn.relu),
        # tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation=tf.nn.relu),
        # tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=tf.keras.losses.MeanSquaredError())
    return model

In [4]:
dnn = build_and_compile_model()
dnn.build(X_train.shape)
dnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (484576, 16)              944       
                                                                 
 dense_1 (Dense)             (484576, 1)               17        
                                                                 
Total params: 961
Trainable params: 961
Non-trainable params: 0
_________________________________________________________________


In [5]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint_filepath = './dnn_model_checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
history = dnn.fit(x=X_train.astype('float32'), 
                  y=y_train.astype('float32'),
                  epochs=100,
                  validation_data=(X_val.astype('float32'), y_val.astype('float32')), 
                  callbacks=[early_stopping_callback, model_checkpoint_callback])

Epoch 1/100


2022-11-29 16:42:43.068421: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


15143/15143 [==============================] - 25s 2ms/step - loss: 0.4788 - val_loss: 0.3406
Epoch 2/100
15143/15143 [==============================] - 27s 2ms/step - loss: 0.4671 - val_loss: 0.3409
Epoch 3/100
15143/15143 [==============================] - 25s 2ms/step - loss: 0.4670 - val_loss: 0.3384
Epoch 4/100
15143/15143 [==============================] - 26s 2ms/step - loss: 0.4667 - val_loss: 0.3396
Epoch 5/100
 2611/15143 [====>.........................] - ETA: 14s - loss: 0.4659

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    # plt.ylim([0.2, 1])
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history=history)

In [ ]:
dnn.load_weights(checkpoint_filepath)

In [ ]:
dnn.evaluate(X_test.astype('float32'), y_test.astype('float32'))

In [ ]:
y_pred = dnn.predict(X_test.astype('float32')).flatten()

In [ ]:
error = []
for i in range(len(y_pred)):
    error.append(y_pred[i] - y_test[i, 0])

In [ ]:
plt.hist(error, bins=25)
plt.xlabel('Prediction Error')
_ = plt.ylabel('Count')

In [ ]:
from metrics import MAE

In [ ]:
MAE(predictions=y_pred, labels=y_test)